In [36]:
import os
import pandas as pd
from sqlalchemy import create_engine


postgres_host = '40.89.151.14' #os.environ['APP_POSTGRES_HOST']
database = 'postgres'
port = '5433'
user = 'mai'
password = '1930'

engine = create_engine(
    f'postgresql://{user}:{password}@{postgres_host}:{port}/{database}'
)

sql_str = """
SELECT
    date,
    subsite_title,
    SUM(content_watch)::float / SUM(content_impression)::float as conversion
FROM (
    SELECT
        user_id_for_mai,
        content_id,
        subsite_title,
        to_char(to_timestamp(rocket_datetime), 'YYYY-MM-DD') as date,
        MAX(CASE WHEN name in ('page_impression', 'click') THEN 1 ELSE 0 END) as content_impression,
        MAX(CASE WHEN name='content_watch' THEN 1 ELSE 0 END) as content_watch
    FROM movie.events
    WHERE
        content_id > 0
        AND subsite_title in ('xboxOne', 'Windows 10')
    GROUP BY user_id_for_mai, content_id, date, subsite_title
) as content_watches
WHERE
    content_impression > 0
GROUP BY date, subsite_title
ORDER BY date, subsite_title
"""

conversion_df = pd.read_sql(sql_str, engine)

conversion_df.head()

print(conversion_df.conversion)

0      0.197802
1      0.200704
2      0.287532
3      0.270677
4      0.439331
         ...   
179    0.374055
180    0.423216
181    0.383515
182    0.482902
183    0.365867
Name: conversion, Length: 184, dtype: float64


In [52]:
win_conversion_sum = 0
xbox_conversion_sum = 0
for i in conversion_df[conversion_df['subsite_title'] == 'Windows 10'].conversion:
    win_conversion_sum = win_conversion_sum + i
    
for j in conversion_df[conversion_df['subsite_title'] == 'xboxOne'].conversion:
    xbox_conversion_sum = xbox_conversion_sum + j


win_conversion = win_conversion_sum/len(conversion_df[conversion_df['subsite_title'] == 'Windows 10'])
xbox_conversion = xbox_conversion_sum/len(conversion_df[conversion_df['subsite_title'] == 'xboxOne']) 

print(win_conversion, xbox_conversion)

0.4174715763462347 0.33164119584125995


In [55]:
import math
from scipy.stats import norm

n1_click = len(conversion_df[conversion_df['subsite_title'] == 'Windows 10'])

n2_click = len(conversion_df[conversion_df['subsite_title'] == 'xboxOne']) 

# ctr
p1 = win_conversion
p2 = xbox_conversion

p = ((p1*n1_click + p2*n2_click)/(n1_click + n2_click))

# значение статистики критерия
T_x = ((p1-p2)/math.sqrt(p*(1-p)*((1/n1_click) + (1/n2_click))))

# квантиль уровня 0.95 нормального распределения
z_095 = norm.ppf(0.95)

print("Верная ли нулевая гипотеза о неравенстве долей: {}".format(T_x > z_095))



Верная ли нулевая гипотеза о равенстве долей: False
